# Script that creates tables for the paper

## Table containing best equation for each data set

In [2]:
# classic imports
import numpy as np
import os
import pickle
import pandas as pd

# this is a weird fix for some weird error with paths
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

In [39]:
# root folder with the results
#results_folder = "../results/2024-06-07-full-results"
results_folder = "../results/2024-06-17-full-results/results-102"
pk_file_name = "symbolic_regression_cp.pk"

# names of the two fitness functions
fitness_coverage = "coverage"
fitness_median = "median"

# data structures
datasets_to_files = {}

# recursively walk through all the folders, looking for files
for folder, subfolders, files in os.walk(results_folder) :
    if pk_file_name in files :
        datasets_to_files[os.path.basename(folder)] = os.path.join(folder, pk_file_name)

print("Number of files: %d" % len(datasets_to_files))
print(datasets_to_files)

Number of files: 22
{'abalone': '../results/2024-06-17-full-results/results-102\\abalone\\symbolic_regression_cp.pk', 'airfoil_self_noise': '../results/2024-06-17-full-results/results-102\\airfoil_self_noise\\symbolic_regression_cp.pk', 'brazilian_houses': '../results/2024-06-17-full-results/results-102\\brazilian_houses\\symbolic_regression_cp.pk', 'california_housing': '../results/2024-06-17-full-results/results-102\\california_housing\\symbolic_regression_cp.pk', 'cars': '../results/2024-06-17-full-results/results-102\\cars\\symbolic_regression_cp.pk', 'concrete_compressive_strength': '../results/2024-06-17-full-results/results-102\\concrete_compressive_strength\\symbolic_regression_cp.pk', 'fifa': '../results/2024-06-17-full-results/results-102\\fifa\\symbolic_regression_cp.pk', 'grid_stability': '../results/2024-06-17-full-results/results-102\\grid_stability\\symbolic_regression_cp.pk', 'health_insurance': '../results/2024-06-17-full-results/results-102\\health_insurance\\symbolic

In [40]:
# right now, the equations are encoded as sympy objects; sympy objects have a method that can
# convert them to latex expressions; so, we could manipulate the expression, replacing symbols
# with more human-readable symbols. But to do that, we need to build a dictionary of Symbols

# and maybe it's actually easier to replace stuff directly inside the latex expression
dict_replacement = {}
dict_replacement[r"x_{0}"] = r"\hat{y}" # x_0 is actually the predicted (normalized) value
dict_replacement[r"x_{1}"] = r"\sigma_d" # x_1 is the sigmas, based on distance
dict_replacement[r"x_{2}"] = r"\sigma_{std}" # x_2 is the sigmas, based on standard deviation
dict_replacement[r"x_{3}"] = r"\sigma_{oob}" # x_3 is the sigmas, based on oob residuals
dict_replacement[r"x_{4}"] = r"\sigma_{var}" # x_4 is the sigmas, based on variance of predictors in ensemble

# all other values are features, so we can just replace them with x_{n-5}
# 116 is the maximum number of features appearing among the considered data sets
dict_replacement_features = {}
for i in range(0, 116) :
    dict_replacement_features["x_{%d}" % (i+5)] = "x_{%d}" % i

In [41]:
# new: add a part that also checks whether the approach is dominated or the only one
# and colors the cells corresponding to that data set accordingly
from check_pareto_optimality import is_dominated # local library

# utility function, to avoid cutting and pasting the same code everywhere
def check_dominance_on_data_set(df, dataset_name, fitness_1, fitness_2) :
    # get the data set row with that particular data set name
    row = df[df["dataset_name"] == dataset_name].iloc[0]
    fitness_columns = [c for c in df.columns if c.endswith(fitness_1) or c.endswith(fitness_2)]
    
    # get the methods names and the associated fitness columns
    methods = dict()
    for fc in fitness_columns :
        if fc.find(fitness_1) != -1 :
            method = fc[:-len(fitness_1)-1]
        elif fc.find(fitness_2) != -1 :
            method = fc[:-len(fitness_2)-1]

        if method not in methods :
            methods[method] = []
        methods[method].append(fc)
        
    # generate points to be compared with the Pareto dominance, and check them 
    non_dominated_methods = []
    for method in methods :
        method_point = (row[methods[method][0]], row[methods[method][1]])
        other_points = [(row[methods[other_method][0]], row[methods[other_method][1]]) 
                        for other_method in methods if other_method != method]

        is_method_point_dominated = False
        for point in other_points :
            if is_dominated(method_point, point) :
                is_method_point_dominated = True
                
        if is_method_point_dominated == False :
            non_dominated_methods.append(method)
    
    # debugging
    #print(dataset_name + ": " + str(non_dominated_methods))
    return non_dominated_methods

# we also need to read the file with all the results
df_results = pd.read_csv(os.path.join(results_folder, "results.csv"))

# this part takes for granted that we have a total of 22 data sets,
# so that we can create two columns of 11 data sets
latex_table = r"\begin{tabular}{l|l|l|l}" + "\n" 
latex_table += r"\textbf{Data set name} & \textbf{Best equation} & \textbf{Data set name} & \textbf{Best equation}\\ \hline \hline" + "\n"

# list of keys in the dictionary
datasets = [key for key in datasets_to_files]

for i in range(0, 11) :
    # columns to the left
    dataset_left = datasets[i]
    # first, check if the approach is dominated
    non_dominated_methods = check_dominance_on_data_set(df_results, dataset_left, fitness_coverage, fitness_median)
    # set cell color
    left_cell_color = ""
    if "symbolic_regression_cp" not in non_dominated_methods :
        left_cell_color = r"\cellcolor[HTML]{ee82ee}"
    elif "symbolic_regression_cp" in non_dominated_methods and len(non_dominated_methods) == 1 :
        left_cell_color = r"\cellcolor[HTML]{ccff00}"
    # add data set name
    # for the data set name, we need to escape all "_" with "\_"
    latex_table += left_cell_color + r"\texttt{" + dataset_left.replace("_", r"\_") + r"} & "
    # add equation
    sr_left = pickle.load(open(datasets_to_files[dataset_left], "rb"))
    equation = sr_left.latex()
    # replacement of variables in two steps, to avoid issues with x_%d
    for key, value in dict_replacement.items() :
        equation = equation.replace(key, value)
    for key, value in dict_replacement_features.items() :
        equation = equation.replace(key, value)
    latex_table += left_cell_color + r"$" + equation + r"$" + r" & "
    
    # columns to the right
    dataset_right = datasets[i+11]
    # first, check if the approach is dominated
    non_dominated_methods = check_dominance_on_data_set(df_results, dataset_right, fitness_coverage, fitness_median)
    # set cell color
    right_cell_color = ""
    if "symbolic_regression_cp" not in non_dominated_methods :
        right_cell_color = r"\cellcolor[HTML]{ee82ee}"
    elif "symbolic_regression_cp" in non_dominated_methods and len(non_dominated_methods) == 1 :
        right_cell_color = r"\cellcolor[HTML]{ccff00}"
    # add data set name
    # for the data set name, we need to escape all "_" with "\_"
    latex_table += right_cell_color + r"\texttt{" + dataset_right.replace("_", r"\_") + r"} & "
    # add equation
    sr_right = pickle.load(open(datasets_to_files[dataset_right], "rb"))
    equation = sr_right.latex()
    # replacement of variables in two steps, to avoid issues with x_%d
    for key, value in dict_replacement.items() :
        equation = equation.replace(key, value)
    for key, value in dict_replacement_features.items() :
        equation = equation.replace(key, value)
    latex_table += right_cell_color + r"$" + equation + r"$" + r" \\ \hline" + "\n"

latex_table += r"\end{tabular}" + "\n"

print(latex_table)

\begin{tabular}{l|l|l|l}
\textbf{Data set name} & \textbf{Best equation} & \textbf{Data set name} & \textbf{Best equation}\\ \hline \hline
\texttt{abalone} & $e^{\sigma_{oob}}$ & \texttt{miami\_housing} & $\sigma_d + 4.96 \sigma_{var}$ \\ \hline
\texttt{airfoil\_self\_noise} & $\sigma_d + \sigma_{var} + 0.141$ & \texttt{Moneyball} & $\sin{\left(0.971 \sin{\left(0.0525 x_{4} + \sin{\left(\cos{\left(\cos{\left(- \hat{y} + x_{3} + x_{4} \right)} \right)} \right)} \right)} \right)}$ \\ \hline
\cellcolor[HTML]{ee82ee}\texttt{brazilian\_houses} & \cellcolor[HTML]{ee82ee}$0.794 + \frac{0.0106 \left(\hat{y} + x_{8}\right)}{\sigma_{std}}$ & \texttt{physiochemical\_protein} & $\sigma_{var} + 0.898$ \\ \hline
\cellcolor[HTML]{ee82ee}\texttt{california\_housing} & \cellcolor[HTML]{ee82ee}$\sigma_{var} + 0.742$ & \texttt{pumadyn32nh} & $0.0495 x_{25} + 1.15$ \\ \hline
\texttt{cars} & $\sigma_{oob} + 0.0761$ & \texttt{QSAR\_fish\_toxicity} & $\sigma_d + e^{\sigma_{std} \sin{\left(2.26 x_{1} \right)}

## Table with longer comparison between methods
So, this table will be 22 (data sets) x 2 (criteria); x 7 (methods). In each row, we report if a method is the best (+), the worst (-) or neither (.) for each of the two criteria

In [28]:
# beginning of the table and header
latex_table = r"\begin{tabular}{l|l|l|l|l|l|l|l|l}" + "\n" 
latex_table += r"\textbf{Data set name} & \textbf{Metric} & \textbf{$SCP$} & \textbf{$NCP_d$} & \textbf{$NCP_{std}$}" 
latex_table += r" & \textbf{$NCP_{oob}$} & \textbf{$NCP_{var}$} & \textbf{$MCP$} & \textbf{$SRCP$} \\ \hline \hline" + "\n"

# load CSV with results
df = pd.read_csv(os.path.join(results_folder, "results.csv"))
# sort rows by data set name
df = df.sort_values(by="dataset_name", key=lambda col: col.str.lower()) # compare lowercase letters
# select columns with fitness values
columns_coverage = [c for c in df.columns if c.endswith(fitness_coverage)]
columns_median = [c for c in df.columns if c.endswith(fitness_median)]
# get the names of the different methods, just to check that we are in the good order
method_names = [ c[:-len(fitness_median)-1] for c in columns_median if c.endswith(fitness_median)]
#print("Methods:", method_names) # looks correct, we can just pick them in order

for index, row in df.iterrows() :
    # get the name of the data set
    dataset_name = row["dataset_name"]
    # escape it and put it in the correct latex format
    latex_table += r"\multirow{2}{*}{"
    latex_table += r"\texttt{" + dataset_name.replace("_", r"\_") + r"}} "
    
    # first sub-row: coverage; find the best (highest) value
    latex_table += "& Coverage "
    coverage_values = row[columns_coverage].values
    #print(coverage_values)
    # get the index(es) of the highest value(s)
    coverage_best = np.where(coverage_values == np.max(coverage_values))[0]
    coverage_worst = np.where(coverage_values == np.min(coverage_values))[0]
    # iterate over the columns, and plot the appropriate stuff
    for i in range(0, len(method_names)) :
        latex_table += r"& "
        # if it's the best or the worst, color it appropriately
        if i in coverage_best :
            latex_table += r"\cellcolor[HTML]{ccff00} "
        elif i in coverage_worst :
            latex_table += r"\cellcolor[HTML]{ee82ee}"
        latex_table += "%.4f" % coverage_values[i] + " "
        
    # second sub-row: median
    latex_table += r"\\ \cline{2-9} " + "\n"
    latex_table += r" & Median "
    median_values = row[columns_median].values
    # get the index(es) of the best (lowest) and worst (highest) values
    median_best = np.where(median_values == np.min(median_values))[0]
    median_worst = np.where(median_values == np.max(median_values))[0]
    # iterate over the columns, and plot the appropriate stuff
    for i in range(0, len(method_names)) :
        latex_table += r"& "
        if i in median_best :
            latex_table += r"\cellcolor[HTML]{ccff00} "
        elif i in median_worst :
            latex_table += r"\cellcolor[HTML]{ee82ee}"
        latex_table += "%.4f" % median_values[i] + " "
        
    # on to the next row
    latex_table += r"\\ \hline \hline" + "\n"

# end of the table
latex_table += r"\end{tabular}" + "\n"
print(latex_table)

\begin{tabular}{l|l|l|l|l|l|l|l|l}
\textbf{Data set name} & \textbf{Metric} & \textbf{$SCP$} & \textbf{$NCP_d$} & \textbf{$NCP_{std}$} & \textbf{$NCP_{oob}$} & \textbf{$NCP_{var}$} & \textbf{$MCP$} & \textbf{$SRCP$} \\ \hline \hline
\multirow{2}{*}{\texttt{abalone}} & Coverage & 0.9474 & 0.9502 & 0.9512 & 0.9455 & \cellcolor[HTML]{ee82ee}0.9311 & 0.9483 & \cellcolor[HTML]{ccff00} 0.9522 \\ \cline{2-9} 
 & Median & 2.9545 & 3.2021 & \cellcolor[HTML]{ee82ee}3.6647 & 3.3293 & 2.7972 & 2.6755 & \cellcolor[HTML]{ccff00} 2.6284 \\ \hline \hline
\multirow{2}{*}{\texttt{airfoil\_self\_noise}} & Coverage & 0.9362 & 0.9388 & \cellcolor[HTML]{ccff00} 0.9468 & 0.9441 & \cellcolor[HTML]{ee82ee}0.8989 & 0.9282 & 0.9441 \\ \cline{2-9} 
 & Median & 1.1607 & 1.4426 & \cellcolor[HTML]{ee82ee}1.6024 & 1.5573 & 1.0324 & 1.1157 & \cellcolor[HTML]{ccff00} 0.9472 \\ \hline \hline
\multirow{2}{*}{\texttt{brazilian\_houses}} & Coverage & 0.9577 & 0.9562 & 0.9581 & \cellcolor[HTML]{ee82ee}0.9540 & 0.9600 & \cel

In [18]:
array = [1, 2, 3, 1, 3, 3]
print(np.argmax(array, keepdims=True))

[2]
